# ИИ в Colab

Colab упрощает интеграцию мощных возможностей генеративного ИИ в ваши проекты. Мы запускаем публичную предварительную версию простой и интуитивно понятной библиотеки Python (google.colab.ai), позволяющей получать доступ к передовым языковым моделям непосредственно в средах Colab. Все пользователи имеют бесплатный доступ к большинству популярных языковых моделей, а платные пользователи получают доступ к более широкому выбору моделей. Это означает, что пользователи могут тратить меньше времени на настройку и установку и больше времени на воплощение своих идей в жизнь. Всего несколькими строками кода вы теперь можете выполнять множество задач:
- Генерировать текст
- Переводить языки
- Создавать креативный контент
- Классифицировать текст

Бесплатный Colab
~50–100 запросов в день
~100K–200K токенов/день

**Не загружайте конфеденциальные данные!!!!**

## Базовые операции

In [ ]:
# @title List available models
from google.colab import ai

ai.list_models()

Выбор модели. Названия моделей дают представление об их возможностях и предполагаемом использовании:

Pro: Это наиболее функциональные модели, идеально подходящие для сложных логических рассуждений, творческих задач и детального анализа.

Flash: Эти модели оптимизированы для высокой скорости и эффективности, что делает их отличными для суммаризации, чат-приложений и задач, требующих быстрой реакции.

Gemma: Это легкие, открытые модели, подходящие для различных задач генерации текста и отлично подходящие для экспериментов.

In [ ]:
# @title Simple batch generation example
# Only text-to-text input/output is supported
from google.colab import ai

response = ai.generate_text("В каком году была принята конституция РФ?")
print(response)

In [ ]:
# @title Choose a different model
from google.colab import ai

response = ai.generate_text("What is the capital of England", model_name='google/gemini-2.5-flash-lite')
print(response)

Для генерации длинных текстовых файлов можно использовать потоковую передачу ответа. В этом случае выходные токены отображаются по мере их генерации, а не дожидаются завершения генерации всего ответа. Это обеспечивает более интерактивный и быстрый отклик. Чтобы включить эту функцию, просто установите параметр stream=True.

In [ ]:
# @title Simple streaming example
from google.colab import ai

stream = ai.generate_text("Подготовь мне лекцию о природе Дальнего Востока", stream=True)
for text in stream:
  print(text, end='')

## Особое форматирование

LineWrapper — умное форматирование текста с автоматическим переносом строк по заданной максимальной ширине (по умолчанию 80 символов),

In [ ]:
#@title Text formatting setup
#code is not necessary for colab.ai, but is useful in fomatting text chunks
import sys

class LineWrapper:
    def __init__(self, max_length=80):
        self.max_length = max_length
        self.current_line_length = 0

    def print(self, text_chunk):
        i = 0
        n = len(text_chunk)
        while i < n:
            start_index = i
            while i < n and text_chunk[i] not in ' \n': # Find end of word
                i += 1
            current_word = text_chunk[start_index:i]

            delimiter = ""
            if i < n: # If not end of chunk, we found a delimiter
                delimiter = text_chunk[i]
                i += 1 # Consume delimiter

            if current_word:
                needs_leading_space = (self.current_line_length > 0)

                # Case 1: Word itself is too long for a line (must be broken)
                if len(current_word) > self.max_length:
                    if needs_leading_space: # Newline if current line has content
                        sys.stdout.write('\n')
                        self.current_line_length = 0
                    for char_val in current_word: # Break the long word
                        if self.current_line_length >= self.max_length:
                            sys.stdout.write('\n')
                            self.current_line_length = 0
                        sys.stdout.write(char_val)
                        self.current_line_length += 1
                # Case 2: Word doesn't fit on current line (print on new line)
                elif self.current_line_length + (1 if needs_leading_space else 0) + len(current_word) > self.max_length:
                    sys.stdout.write('\n')
                    sys.stdout.write(current_word)
                    self.current_line_length = len(current_word)
                # Case 3: Word fits on current line
                else:
                    if needs_leading_space:
                        # Define punctuation that should not have a leading space
                        # when they form an entire "word" (token) following another word.
                        no_leading_space_punctuation = {
                            ",", ".", ";", ":", "!", "?",        # Standard sentence punctuation
                            ")", "]", "}",                     # Closing brackets
                            "'s", "'S", "'re", "'RE", "'ve", "'VE", # Common contractions
                            "'m", "'M", "'ll", "'LL", "'d", "'D",
                            "n't", "N'T",
                            "...", "…"                          # Ellipses
                        }
                        if current_word not in no_leading_space_punctuation:
                            sys.stdout.write(' ')
                            self.current_line_length += 1
                    sys.stdout.write(current_word)
                    self.current_line_length += len(current_word)

            if delimiter == '\n':
                sys.stdout.write('\n')
                self.current_line_length = 0
            elif delimiter == ' ':
                # If line is full and a space delimiter arrives, it implies a wrap.
                if self.current_line_length >= self.max_length:
                    sys.stdout.write('\n')
                    self.current_line_length = 0

        sys.stdout.flush()


In [ ]:
# @title Formatted streaming example
from google.colab import ai

wrapper = LineWrapper()
for chunk in ai.generate_text('Подготовь доклад о римской империи.', model_name='google/gemini-2.5-flash-lite', stream=True):
  wrapper.print(chunk)

## Организация диалога

In [ ]:
user_question = input("Введите ваш вопрос: ")
print(f"Вы спросили: {user_question}")

# Пример использования с генерацией текста
from google.colab import ai
response = ai.generate_text(user_question)
print("\nОтвет:", response)

In [ ]:
from google.colab import ai
import ipywidgets as widgets
from IPython.display import display, clear_output

output = widgets.Output()
text_input = widgets.Text(
    description='Вопрос:',
    placeholder='Введите вопрос здесь...',
    layout=widgets.Layout(width='80%')
)

def on_submit(sender):
    with output:
        clear_output()
        print(f" Вопрос: {text_input.value}")
        print("Генерация ответа...")
        response = ai.generate_text(text_input.value)
        clear_output()
        print(f"Вопрос: {text_input.value}")
        print(f"Ответ: {response}\n")

text_input.on_submit(on_submit)
display(text_input, output)

Чтобы организовать диалог в Google Colab с «памятью» (сохранением контекста предыдущих реплик), важно понимать: ai.generate_text() сам по себе НЕ запоминает историю — каждый вызов независим. Чтобы модель «помнила» разговор, нужно вручную передавать историю в промпт.

In [ ]:
from google.colab import ai
import ipywidgets as widgets
from IPython.display import display, clear_output

history = []
chat_output = widgets.Output(layout={'border': '1px solid #ddd', 'height': '300px', 'overflow': 'auto'})
input_box = widgets.Text(description='Вы:', placeholder='Введите сообщение...', layout=widgets.Layout(width='80%'))

def on_submit(sender):
    question = input_box.value.strip()
    if not question:
        return

    # Формируем промпт с контекстом
    prompt = "Ты — дружелюбный ассистент. Отвечай естественно.\n\n"
    for role, text in history[-6:]:  # Берём последние 3 пары реплик (ограничение длины)
        prompt += f"{role}: {text}\n"
    prompt += f"Пользователь: {question}\nАссистент:"

    # Генерация
    with chat_output:
        print(f"Вы: {question}")
        print("ИИ думает...", end="\r")
        response = ai.generate_text(prompt)
        print(" " * 30, end="\r")  # Стираем "думает"
        print(f"ИИ: {response}\n")

    # Сохраняем в историю
    history.append(("Пользователь", question))
    history.append(("Ассистент", response))
    input_box.value = ""

input_box.on_submit(on_submit)
display(input_box, chat_output)

## Обработка данных

In [ ]:
from google.colab import ai

table = """
Город,Население,Страна
Москва,12500000,Россия
Париж,2160000,Франция
Токио,13900000,Япония
"""

prompt = f"""Проанализируй таблицу и ответь на вопрос.

Таблица:
{table}

Вопрос: Какой город самый густонаселённый?"""

response = ai.generate_text(prompt)
print(response)

In [ ]:
from google.colab import files, ai
import pandas as pd

# 1. Загружаем файл
uploaded = files.upload()  # Откроется окно выбора файла

# 2. Читаем таблицу
df = pd.read_csv(list(uploaded.keys())[0])

# 3. Преобразуем в текст (лучше — в формате Markdown для читаемости)
table_text = df.to_markdown(index=False)

# 4. Запрашиваем анализ
prompt = f"""Таблица с данными:

{table_text}

Вопрос: Сколько всего строк в таблице и какие столбцы присутствуют?

Расчитатай все возможные статсические характеристики.
"""

response = ai.generate_text(prompt)
print(response)

In [ ]:
import pandas as pd
from google.colab import ai

# 1. Загружаем и анализируем данные программно
df = pd.read_csv('data.csv')
stats = df.describe().to_markdown()

# 2. Передаём агрегированные данные + вопрос модели
prompt = f"""Статистика по таблице:

{stats}

Вопрос: Какие аномалии ты заметил в распределении данных? Дай рекомендации."""

response = ai.generate_text(prompt)
print(response)